In [24]:
import pickle

### Read data

In [25]:
data_path = "data.pkl"

with open(data_path, "rb") as f:
    dataset = pickle.load(f)

Check the dataset:

'description' is for text descripton of an object

'code' is a cadquery code

In [26]:
len(dataset)

21

In [27]:
dataset[0]

{'description': 'Start by creating a new coordinate system with Euler angles set to 0, 0, and -90 degrees, and a translation vector of 0, 0.4218, and 0. On the first face, draw a closed loop consisting of 4 lines. Begin the first line at the origin (0, 0) and extend it to (0.75, 0). Continue with the second line from (0.75, 0) to (0.75, 0.75), then the third line from (0.75, 0.75) to (0, 0.75), and finally, the fourth line back to the origin. Scale the entire 2-dimensional sketch by a factor of 0.75. Next, transform the scaled sketch into 3 dimensions by rotating it with the same Euler angles used for the coordinate system and translating it with the same vector. Extrude the transformed sketch 0.4218 units along the normal direction to create a solid body. Ensure that the extrusion does not occur in the opposite direction of the normal. The final dimensions of the first part will be approximately 0.75 units in length, 0.75 units in width, and 0.4218 units in height, forming the base of

In [28]:
dataset

[{'description': 'Start by creating a new coordinate system with Euler angles set to 0, 0, and -90 degrees, and a translation vector of 0, 0.4218, and 0. On the first face, draw a closed loop consisting of 4 lines. Begin the first line at the origin (0, 0) and extend it to (0.75, 0). Continue with the second line from (0.75, 0) to (0.75, 0.75), then the third line from (0.75, 0.75) to (0, 0.75), and finally, the fourth line back to the origin. Scale the entire 2-dimensional sketch by a factor of 0.75. Next, transform the scaled sketch into 3 dimensions by rotating it with the same Euler angles used for the coordinate system and translating it with the same vector. Extrude the transformed sketch 0.4218 units along the normal direction to create a solid body. Ensure that the extrusion does not occur in the opposite direction of the normal. The final dimensions of the first part will be approximately 0.75 units in length, 0.75 units in width, and 0.4218 units in height, forming the base o

Additional function for executing the code in a string format and create a cadquery skecth

In [29]:
import cadquery as cq
from io import StringIO
import contextlib


def execute_cadquery_code(code_string):
    safe_globals = {"cq": cq}

    output = StringIO()
    try:
        with contextlib.redirect_stdout(output):
            exec(code_string, safe_globals)

            if "r" in safe_globals and isinstance(safe_globals["r"], cq.Workplane):
                return safe_globals["r"], output.getvalue()

        return None, "No valid CadQuery object named 'r' found"
    except Exception as e:
        return None, f"Error: {str(e)}"

### Display first 5 figures

In [30]:
# # for i in range(min(5, len(dataset))):  # len(dataset)):
# for i in range(5, 10):  # len(dataset)):
#     model, _ = execute_cadquery_code(dataset[i]["code"])

#     display(model)

In [31]:
indices = [6, 7, 8]

## Prompt construction

### Base

In [47]:
idx = 6

code, description = dataset[idx]["code"], dataset[idx]["description"]

model = execute_cadquery_code(code)[0]

print(f"##Code for index {idx}:\n{code}\n\n##Description:\n{description}\n")
# model, _ = execute_cadquery_code(dataset[indices[idx]]["code"])
model

##Code for index 6:
import cadquery as cq
w0=cq.Workplane('ZX',origin=(-58,0,0))
w1=cq.Workplane('ZX',origin=(-33,-100,0))
r=w0.sketch().face(w0.sketch().push([(0,58)]).circle(58)).finalize().extrude(-100,both=True).cut(w1.sketch().face(w1.sketch().push([(0,33)]).circle(33)).finalize().extrude(200))

##Description:
Start by constructing a cylindrical object with a flat circular top and bottom. First, create a new coordinate system with Euler angles set to 0, 0, -90 degrees and a translation vector of 0, 0.375, 0. Next, draw a 2-dimensional sketch on a new face. Within this face, draw a single loop and within the loop, draw a circle centered at (0.2188, 0.2188) with a radius of 0.2188. Apply a scale factor of 0.4375 to the sketch. Rotate and translate the scaled sketch using the previously defined coordinate system. Extrude the sketch 0.1875 units in both the normal and opposite directions to create a solid body. The dimensions of this first part are 0.4375 in length, 0.4375 in width, a

In [40]:
basic_p = """
You are an expert in parametric 3D modeling using CadQuery and Python. Your task is to write a Python code using the CadQuery library that generates a 3D model matching a reference information about the shape as closely as possible.

Requirements:
1. The code must use CadQuery primitives and operations.
2. The code should assign final CadQuery solid object (cq.Workplane with 3D geometry) to variable 'r'.
3. The script must be executable in a standard Python environment with CadQuery installed (no other packages).
4. Do not include any comments  to the code.

You have a text shape description with important information.
Shape description:\n
"""

In [41]:
pos_description = """
\n\nPositioning:
Suppose following coordinate system:
- vector OX points to the right (+X direction)
- vector OY points up (+Y direction)
- vector OZ points towards viewer (+Z direction)

Then the cylinder should be positioned as follows:
- Plane +Z corresponds to the side (longest) face of the cylinder
- Plane +Y corresponds to the circular face of the cylinder
"""

### Dataset prompt

In [36]:
# Gemini initial prompt

initial_p = basic_p + description + pos_description
print(initial_p)


You are an expert in parametric 3D modeling using CadQuery and Python. Your task is to write a Python code using the CadQuery library that generates a 3D model matching a reference information about the shape as closely as possible.

Requirements:
1. The code must use CadQuery primitives and operations.
2. The code should assign final CadQuery solid object (cq.Workplane with 3D geometry) to variable 'r'.
3. The script must be executable in a standard Python environment with CadQuery installed (no other packages).
4. Do not include any comments  to the code.

You have a text shape description with important information.
Shape description:

Start by constructing a cylindrical object with a flat circular top and bottom. First, create a new coordinate system with Euler angles set to 0, 0, -90 degrees and a translation vector of 0, 0.375, 0. Next, draw a 2-dimensional sketch on a new face. Within this face, draw a single loop and within the loop, draw a circle centered at (0.2188, 0.2188) 

In [39]:
initial_p_code = """
import cadquery as cq

r = cq.Workplane("XZ")
r = r.circle(0.4375 / 2.0)
r = r.extrude(0.375, both=True)
r = r.faces(">Y").workplane()
r = r.hole(diameter=0.25, depth=0.75)
"""

initial_p_model = execute_cadquery_code(initial_p_code)[0]


initial_p_model

### Simplified prompt

In [42]:
simplified_desc = """
A solid cylinder with a diameter of 0.4375 and a height of 0.375 with a cylindrical cut with a diameter of 0.25 and a depth of 0.75
"""

In [45]:
# Gemini simplified prompt

simp_p = basic_p + simplified_desc + pos_description
print(simp_p)


You are an expert in parametric 3D modeling using CadQuery and Python. Your task is to write a Python code using the CadQuery library that generates a 3D model matching a reference information about the shape as closely as possible.

Requirements:
1. The code must use CadQuery primitives and operations.
2. The code should assign final CadQuery solid object (cq.Workplane with 3D geometry) to variable 'r'.
3. The script must be executable in a standard Python environment with CadQuery installed (no other packages).
4. Do not include any comments  to the code.

You have a text shape description with important information.
Shape description:


A solid cylinder with a diameter of 0.4375 and a height of 0.375 with a cylindrical cut with a diameter of 0.25 and a depth of 0.75



Positioning:
Suppose following coordinate system:
- vector OX points to the right (+X direction)
- vector OY points up (+Y direction)
- vector OZ points towards viewer (+Z direction)

Then the cylinder should be posi

In [46]:
simp_p_code = """
import cadquery as cq

r = (
    cq.Workplane("XZ")
    .circle(0.4375 / 2)
    .extrude(0.375)
    .faces(">Y")
    .hole(0.25)
)
"""

simp_p_model = execute_cadquery_code(simp_p_code)[0]


simp_p_model